## Libraries & Daily return function

In [83]:
import pandas as pd
import numpy as np
from scipy.stats import norm
import math

In [84]:
stocks = pd.read_csv("Portfolio.csv", index_col = "Date")

In [85]:
# Calculate daily returns
def daily_returns(adjusted_cp):
    r = (adjusted_cp / adjusted_cp.shift(1)) - 1
    return r


## Variance-Covariance Method

In [86]:
def calcuate_VaR_cov_method(portfolio, w, total_portfolio):
    daily_return_df = portfolio.apply(daily_returns).dropna()
    weights = np.array(w)
    portfolio_return = (daily_return_df * weights).sum(axis = 1)
    portfolio_mean = portfolio_return.mean()
    weights = weights.reshape(len(w),1)
    cov_matrix = daily_return_df.cov()
    portfolio_sd = np.sqrt(weights.T @ cov_matrix @ weights)[0][0]
    VaR99 = round((portfolio_mean + norm.ppf(0.01) * portfolio_sd) * total_portfolio,0)
    VaR95 = round((portfolio_mean + norm.ppf(0.05) * portfolio_sd) * total_portfolio,0)
    VaR90 = round((portfolio_mean + norm.ppf(0.10) * portfolio_sd) * total_portfolio,0)
    CVaR99 = round(portfolio_mean - (portfolio_sd / (0.01))*norm.pdf(norm.ppf(0.01)) * total_portfolio,0)
    CVaR95 = round(portfolio_mean - (portfolio_sd / (0.05))*norm.pdf(norm.ppf(0.05)) * total_portfolio,0)
    CVaR90 = round(portfolio_mean - (portfolio_sd / (0.1))*norm.pdf(norm.ppf(0.1)) * total_portfolio,0)
    print("For 1 day trading interval VaR(99%) is $", VaR99)
    print("For 1 day trading interval VaR(95%) is $", VaR95)
    print("For 1 day trading interval VaR(90%) is $", VaR90)
    print("For 1 day trading interval CVaR(99%) is $", CVaR99)
    print("For 1 day trading interval CVaR(95%) is $", CVaR95)
    print("For 1 day trading interval CVaR(90%) is $", CVaR90)
calcuate_VaR_cov_method(stocks, w = [0.3,0.4,0.3], total_portfolio = 1000000)    
    

For 1 day trading interval VaR(99%) is $ -35767.0
For 1 day trading interval VaR(95%) is $ -25073.0
For 1 day trading interval VaR(90%) is $ -19372.0
For 1 day trading interval CVaR(99%) is $ -41821.0
For 1 day trading interval CVaR(95%) is $ -32367.0
For 1 day trading interval CVaR(90%) is $ -27538.0


## Historical Method

In [87]:
def calcuate_VaR_hist_method(portfolio, w, total_portfolio):
    daily_return_df = portfolio.apply(daily_returns).dropna()
    weights = np.array(w)
    portfolio_return = (daily_return_df * weights).sum(axis = 1)
    VaR99 = round(np.percentile(portfolio_return,1) * total_portfolio,0)
    VaR95 = round(np.percentile(portfolio_return,5) * total_portfolio,0)
    VaR90 = round(np.percentile(portfolio_return,10) * total_portfolio,0)
    CVaR99 = round(np.mean(portfolio_return[portfolio_return < np.percentile(portfolio_return,1)]) * total_portfolio,0)
    CVaR95 = round(np.mean(portfolio_return[portfolio_return < np.percentile(portfolio_return,5)]) * total_portfolio,0)
    CVaR90 = round(np.mean(portfolio_return[portfolio_return < np.percentile(portfolio_return,10)]) * total_portfolio,0)
    print("For 1 day trading interval VaR(99%) is $", VaR99)
    print("For 1 day trading interval VaR(95%) is $", VaR95)
    print("For 1 day trading interval VaR(90%) is $", VaR90)
    print("For 1 day trading interval CVaR(99%) is $", CVaR99)
    print("For 1 day trading interval CVaR(95%) is $", CVaR95)
    print("For 1 day trading interval CVaR(90%) is $", CVaR90)
    
calcuate_VaR_hist_method(stocks, w = [0.3,0.4,0.3], total_portfolio = 1000000)

For 1 day trading interval VaR(99%) is $ -35630.0
For 1 day trading interval VaR(95%) is $ -23896.0
For 1 day trading interval VaR(90%) is $ -19206.0
For 1 day trading interval CVaR(99%) is $ -41297.0
For 1 day trading interval CVaR(95%) is $ -31545.0
For 1 day trading interval CVaR(90%) is $ -26954.0


## Monte Carlo Simulation method 

In [88]:
stocks = pd.read_csv("Portfolio.csv", index_col = "Date")
returns = stocks.apply(daily_returns).dropna()
mu = [0.04, 0.04, 0.04]
w = np.array([0.30, 0.40, 0.30])
T = 1 # 1 year
n = 252 # 1000 cuts in 1 year
m = 100 # num sims
p = stocks.shape[1]
s0 = np.array(stocks.iloc[stocks.shape[0]-1])
sigma = np.array(returns.std())
dt = T/n

cor_returns = returns.corr()
cov_returns = returns.cov()
L = np.linalg.cholesky(cor_returns)

simulated_data = np.empty((n+1, m))
for i in range(m):
    correlated_random = (L @ np.random.normal(0, 1, size = (n,p)).T).T
    st = np.exp(
    (mu - sigma**2 / 2) * dt + sigma * correlated_random 
    ) 
    # Each col of st is a particular stock's movement over time
    st = np.vstack([np.ones(p), st])
    st = s0 * st.cumprod(axis = 0)
    portfolio = (st*w).sum(axis = 1)
    simulated_data[:, i] = portfolio.T
sim_list = ["Simulation " + str(i) for i in range (1,101)]
simulated_portfolio_df = pd.DataFrame(simulated_data)
simulated_portfolio_df.columns = sim_list
simulated_portfolio_returns = simulated_portfolio_df.apply(daily_returns).dropna()

In [89]:
total_portfolio = 1000000
var99 = [round(np.percentile(simulated_portfolio_returns.iloc[:,i],1) * total_portfolio,0) for i in range(100)]
var95 = [round(np.percentile(simulated_portfolio_returns.iloc[:,i],5) * total_portfolio,0) for i in range(100)]
var90 = [round(np.percentile(simulated_portfolio_returns.iloc[:,i],10) * total_portfolio,0) for i in range(100)]

Cvar99 = [round(np.mean(simulated_portfolio_returns.iloc[:,i][simulated_portfolio_returns.iloc[:,i] < np.percentile(simulated_portfolio_returns.iloc[:,i],1)]) * total_portfolio,0) for i in range(100)]

Cvar95 = [round(np.mean(simulated_portfolio_returns.iloc[:,i][simulated_portfolio_returns.iloc[:,i] < np.percentile(simulated_portfolio_returns.iloc[:,i],5)]) * total_portfolio,0) for i in range(100)]

Cvar90 = [round(np.mean(simulated_portfolio_returns.iloc[:,i][simulated_portfolio_returns.iloc[:,i] < np.percentile(simulated_portfolio_returns.iloc[:,i],10)]) * total_portfolio,0) for i in range(100)]

In [91]:
def summary_statistics(var):
    return {
        "Mean" : np.mean(var),
        "Max" : max(var),
        "Min" : min(var)
    }

In [92]:
summary_statistics(var99)

{'Max': -25645.0, 'Mean': -33808.83, 'Min': -39126.0}

In [93]:
summary_statistics(var95)

{'Max': -19883.0, 'Mean': -24330.87, 'Min': -29214.0}

In [94]:
summary_statistics(var90)

{'Max': -15633.0, 'Mean': -19029.02, 'Min': -23361.0}

In [95]:
summary_statistics(Cvar90)

{'Max': -21422.0, 'Mean': -25907.74, 'Min': -29897.0}

In [96]:
summary_statistics(Cvar95)

{'Max': -25040.0, 'Mean': -30508.01, 'Min': -34784.0}

In [97]:
summary_statistics(Cvar99)

{'Max': -30330.0, 'Mean': -38537.72, 'Min': -50281.0}